In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv3D, Dense, Dropout, Flatten, InputLayer, Lambda, MaxPool2D, MaxPooling3D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#, MultiGPUModelCheckpoint
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import plot_model
#, multi_gpu_model
import tensorflow as tf

## Hyperparameters and Settings

In [ ]:
size_ResNet50 = (224,224,3)
size_VGG19 = (224,224,3)
size_IncpV3 = (299,299,3)

In [ ]:
BATCH_size = 32
OPT = Adam(learning_rate=0.001) #Adam, SGD, RMSprop, 
LOSS_func = tf.keras.losses.CategoricalCrossentropy() # 
EPOCHS = 20
METRICS = [
      #tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy"),
      #'acc',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
]
weight_decay = 0.001

### **Data Augmentation/ Dataset splitting**

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
                                   rotation_range=5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.1)

In [ ]:
test_datagen  = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.1
                                  )

### Load in the data

In [ ]:
input_size = (224,224)
#input_size = (299,299)

images_train = train_datagen.flow_from_directory(directory = '../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train',
                                                target_size = input_size, 
                                                class_mode = 'categorical',
                                                subset = 'training',
                                                batch_size=BATCH_size)

images_validation = train_datagen.flow_from_directory(directory = '../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train',
                                                target_size = input_size,
                                                class_mode = 'categorical',
                                                subset = 'validation',
                                                batch_size=BATCH_size)

images_test = train_datagen.flow_from_directory(directory = '../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test',
                                                target_size = input_size,
                                                class_mode = 'categorical',
                                                #subset = 'testing'
                                                batch_size=BATCH_size)


# Data Exploration

In [ ]:
#import nibabel as nib
#import numpy as np

In [ ]:
import matplotlib.image as mpimg
import random

## Mild Demented

In [ ]:
plt.figure(figsize=(20,20))
test_folder="../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented" 
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

## Moderate Demented

In [ ]:
plt.figure(figsize=(20,20))
test_folder="../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented" 
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

## Non Demented

In [ ]:
plt.figure(figsize=(20,20))
test_folder="../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented" 
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

## Very Mild Demented

In [ ]:
plt.figure(figsize=(20,20))
test_folder="../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented" 
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

Not ready yet...

# **Transfer Learning Pre-Trained Model (2D CNNs)**

In [ ]:
base_model = VGG19(input_shape=size_VGG19, include_top=False, weights="imagenet")

In [ ]:
#base_model = InceptionV3(input_shape=size_IncpV3, include_top=False, weights='imagenet')

#base_model = ResNet50(input_shape=size_ResNet50, include_top=False, weights='imagenet')

In [ ]:
#Freezing the top layers (pre-trained model)
for each_layer in base_model.layers:
    each_layer.trainable = False

In [ ]:
plot_model(base_model) 

In [ ]:
# Building Model (v3)

model = Sequential()

#model.add(Lambda(lambda image: tf.image.resize(image, input_size)))
model.add(base_model)

model.add(Flatten())

"""
model.add(Dense(256, activation='relu', kernel_regularizer=l2(weight_decay), 
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(256, activation='relu', kernel_regularizer=l2(weight_decay), 
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(256, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
"""
"""
model.add(Dense(256, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(128, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
"""
model.add(Dense(128, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(64, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(weight_decay),
                kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
        #model.add(Lambda(lambda x: tf.abs(x)))
        #model.add(LSTM(256))
        #model.add(LSTM(128))
model.add(Dense(4, activation='softmax'))


In [ ]:
plot_model(model)

In [ ]:
model.summary()

In [ ]:
#input_shape = (None, None, None, 3)

### Model (3D CNNs)

In [ ]:
"""
model = Sequential()

model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))
"""

In [ ]:
#plot_model(model)

### Callbacks and Model Checkpoint

In [ ]:
#Checking if there is more than 1 GPU available (This affects the ModelCheckpoint!)
"""
GPU_available = len(os.getenv("CUDA_VISIBLE_DEVICES", "1").split(","))

if (GPU_available > 1):
    model_train = multi_gpu_model(model, GPU_available)
    checkpoint = MultiGPUModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',
                                         base_model=model)
else:
    model_train = model
    checkpoint = ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', verbose=1)
"""
checkpoint = ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', verbose=1)

In [ ]:
#Callbacks
my_callbacks = [checkpoint] #EarlyStopping(),

In [ ]:
model.compile(optimizer=OPT, loss=LOSS_func, metrics=METRICS)

In [ ]:
history = model.fit(images_train ,validation_data=images_validation, epochs = EPOCHS, verbose = 1, callbacks=my_callbacks)

In [ ]:
scores = model.evaluate(images_test)

In [ ]:
for c in scores:
    print(c)

In [ ]:
print(f"Val_Loss: {scores[0]}\nAccuracy: {scores[1]}\nPrecision: {scores[2]}\nRecall: {scores[3]}\nAUC: {scores[4]}\n")

In [ ]:
"""
import jovian
jovian.commit(project='alzheimer-mri-classif-project')
"""